In [ ]:
import main
index = main.run()

In [ ]:
import logging
from llama_index import ServiceContext
from llama_index.llms import OpenAI

service_context = ServiceContext.from_defaults(llm=OpenAI(model="gpt-3.5-turbo"))
# service_context = ServiceContext.from_defaults(llm=OpenAI(model="gpt-4"))

query_engine = index.as_query_engine(similarity_top_k=5, service_context=service_context, verbose=True)
query_str = "Can you tell me about the key concepts for safety finetuning"
response = query_engine.query(query_str)
logging.info(response)

query_str = "Tell me about LVR"
response = query_engine.query(query_str)
logging.info(response)

query_str = "What plagues current AMM designs?"
response = query_engine.query(query_str)
logging.info(response)

# TODO 2023-09-27: improve the response engine with react agent chatbot.

logging.info(response)
# chat_engine = index.as_chat_engine(chat_mode=ChatMode.REACT, verbose=True)
# response = chat_engine.chat("Hi")
pass

In [48]:
from IPython.display import display, HTML, Javascript
import json

data = {}  # Global variable to store the loaded JSON data

def pretty_print_json(file_path: str):
    global data
    with open(file_path, 'r') as file:
        data = json.load(file)
    formatted_json = json.dumps(data, indent=4)

    # Define a JavaScript function to make JSON collapsible and editable
    js = """
    function createCollapsible(jsonObj, isTopLevel=true, path="") {
        var type = typeof jsonObj;
        if (type === "object") {
            var jsonDiv = document.createElement("div");
            jsonDiv.style.marginLeft = isTopLevel ? "0px" : "20px";
            
            var keys = Object.keys(jsonObj);
            for (var i = 0; i < keys.length; i++) {
                var key = keys[i];
                var value = jsonObj[key];
                
                var itemDiv = document.createElement("div");
                itemDiv.style.borderLeft = "1px solid #ccc";
                itemDiv.style.marginTop = "2px";
                jsonDiv.appendChild(itemDiv);
                
                var keySpan = document.createElement("span");
                keySpan.textContent = key + ": ";
                keySpan.style.fontWeight = "bold";
                itemDiv.appendChild(keySpan);
                
                var childPath = path + "['" + key + "']";
                
                if (typeof value === "object") {
                    keySpan.addEventListener("click", function() {
                        var sibling = this.nextSibling;
                        sibling.style.display = sibling.style.display === "none" ? "block" : "none";
                    });
                    
                    var childDiv = createCollapsible(value, false, childPath);
                    childDiv.style.display = isTopLevel || childPath.endsWith("['response']") ? "block" : "none";
                    itemDiv.appendChild(childDiv);
                } else {
                    var valueSpan = document.createElement("span");
                    valueSpan.textContent = JSON.stringify(value);
                    itemDiv.appendChild(valueSpan);
                    
                    if (childPath.endsWith("['subjective_score']")) {
                        valueSpan.contentEditable = true;
                        valueSpan.addEventListener("blur", function() {
                            var newValue = this.textContent;
                            var pythonCode = `data${childPath} = '\${newValue}'`;
                            IPython.notebook.kernel.execute(pythonCode);
                        });
                    }
                }
            }
            return jsonDiv;
        } else {
            var textDiv = document.createElement("div");
            textDiv.textContent = JSON.stringify(jsonObj);
            return textDiv;
        }
    }
    """
    
    # Create a unique ID for the div element to contain the JSON view
    div_id = f"json-div-{hash(formatted_json)}"
    
    # Create HTML and JS to display JSON interactively
    html = f"<div id='{div_id}'></div>"
    js += f"""
    var jsonDiv = document.getElementById("{div_id}");
    var jsonObj = {formatted_json};
    var collapsibleDiv = createCollapsible(jsonObj);
    jsonDiv.appendChild(collapsibleDiv);
    """
    
    # Display the interactive and collapsible JSON view
    display(HTML(html))
    display(Javascript(js))

def save_json(file_path: str):
    global data
    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)


In [45]:
from utils import root_directory
root_dir = root_directory()
file_path = f'{root_dir}/datasets/evaluation_results/2023-09-29_text-embedding-ada-002_gpt-3.5-turbo_1536_230.json'
pretty_print_json(file_path)

<IPython.core.display.Javascript object>

In [49]:
save_json(file_path)

In [ ]:
i=1
for source_node in response.source_nodes: 
    print(f"Node #{i}:\n \n{source_node}\n\n")
    i+=1